# Stock Trading test

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from elegantrl.train.run_tutorial import *
from elegantrl.train.run_parallel import *
from elegantrl.train.config import Arguments
from elegantrl.agents import AgentPPO, AgentDDPG
from stock_env_multiple import StockEnvMultiple

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [4]:
tics = pd.read_csv('data/filtered_with_ti.csv', header=0)
all_tickers = tics['tic'].unique()
daterange = tics[tics['tic'] == 'AAPL']['Date'].to_numpy()

In [5]:
def check_tickers(stock):
    return stock in all_tickers

def check_date(date):
    return date in daterange

In [ ]:
check_date('2008-03-20')

In [6]:
check_tickers('TSLA')

True

In [ ]:
# Arguments


In [7]:
tickers = ['TSLA']
initial_stocks = np.zeros([len(tickers), 1])


gamma = 0.99
max_stock = 1e2
initial_capital = 1e6
initial_stocks = np.zeros(len(tickers), dtype=np.float32)

start_date = '2006-03-20'
end_date = '2008-03-20'
start_eval_date = '2016-01-01'
end_eval_date = '2021-01-01'

# Agent
ppo = AgentPPO.AgentPPO() # AgentSAC(), AgentTD3(), AgentDDPG()?
ppo.if_use_gae = True
ppo.lambda_entropy = 0.04
ppo.if_on_policy = True

# Environment
env = StockEnvMultiple(daterange, tickers=tickers, begin_date=start_date, end_date=end_date, initial_stocks=initial_stocks)
env.max_step = 100

args = Arguments(env, ppo)


# Hyperparameters
args.gamma = gamma
args.break_step = int(2e5)
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = args.max_step * 4
args.reward_scale = 1e-4
args.batch_size = 2 ** 10
args.repeat_times = 1
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.worker_num = 4
args.thread_num = 16
args.if_allow_break = False
args.target_return = 1.1e7
args.rollout_num = 6 # the number of rollout workers (larger is not always faster)

NameError: name 'tics' is not defined

In [ ]:
a = time.time()
train_and_evaluate_mp(args)
print(time.time() - a)

In [ ]:
args.env.stocks

In [ ]:
start_eval_date = '2016-01-05'
end_eval_date = '2021-01-05'

agent = AgentPPO.AgentPPO() # AgentSAC(), AgentTD3(), AgentDDPG()?
agent.if_use_gae = True
agent.lambda_entropy = 0.04

initial_stocks = np.zeros([len(tickers)]).tolist()
print(initial_stocks)

env_eval = StockEnvMultiple(daterange, tickers=tickers,
                            begin_date=start_eval_date,
                            end_date=end_eval_date, 
                            initial_stocks=initial_stocks)


agent.if_on_policy = False
args = Arguments(env_eval, agent)

args.if_remove = False
args.cwd = './AgentPPO_RLStockEnv-v2_(0,)'
args.gamma = gamma
args.break_step = int(2e5)
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = args.max_step * 4
args.batch_size = 2 ** 10
args.repeat_times = 2 ** 3
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = False
args.target_return = 1.1e7

args.rollout_num = 6 # the number of rollout workers (larger is not always faster)
args.init_before_training()

env_eval.draw_cumulative_return(args, torch)
plt.show()

In [ ]:
plt.show()